### Notebook responsável por buscar os scouts dos jogadores

In [2]:
import requests
import json
import datetime

In [6]:
%run ./database.ipynb
%run ./partidas.ipynb

In [1]:
# Obtendo os dados pela API

def get_dados_rodada(rodada):
    url = 'https://api.cartolafc.globo.com/atletas/pontuados/{}'.format(rodada)
    r = requests.get(url)
    json_dados_rodada = r.json()
    numero_rodada = json_dados_rodada['rodada']
    total_atletas_rodada = json_dados_rodada['total_atletas']
    df_atletas = pd.DataFrame.from_dict(json_dados_rodada['atletas'], orient='index')
    df_atletas['id_atleta'] = df_atletas.index
    json_struct = json.loads(df_atletas.to_json(orient="records"))    
    df_atletas_flat = pd.json_normalize(json_struct)
    df_atletas_flat.fillna(0, inplace=True)
    
    return df_atletas_flat, numero_rodada, total_atletas_rodada

In [ ]:
def scouts_possiveis():
    return ['scout.FC', 'scout.FS', 'scout.PI',\
       'scout.FF', 'scout.A', 'scout.DS', 'scout.DE', 'scout.GS', 'scout.FD',\
       'scout.SG', 'scout.CA', 'scout.FT', 'scout.G', 'scout.PC',\
       'scout.PS', 'scout.PP', 'scout.I', 'scout.DP', 'scout.GC', 'scout.CV']

In [1]:
def get_pontuacao_scouts(scout):
    pontuacao = {
        'scout.G' : ( 8.0, 'Gol'),
        'scout.A' : ( 5.0, 'Assistência'),
        'scout.FT': ( 3.0, 'Finalização na trave'),
        'scout.FD': ( 1.2, 'Finalização defendida'),
        'scout.FF': ( 0.8, 'FInalização para fora'),
        'scout.FS': ( 0.5, 'Falta sofrida'),
        'scout.PS': ( 1.0, 'Pênalti sofrido'),
        'scout.DP': ( 7.0, 'Defesa de pênalti'),
        'scout.SG': ( 5.0, 'Jogo sem sofrer gols'),
        'scout.DE': ( 1.0, 'Defesa'),
        'scout.DS': ( 1.2, 'Desarme'),
        'scout.PP': (-4.0, 'Pênalti perdido para fora'), # A Api do cartola esta considerando -4
        'scout.I' : (-0.1, 'Impedimento'),
        'scout.PI': (-0.1, 'Passe incompleto'),
        'scout.CA': (-1.0, 'Cartão amarelo'),
        'scout.GS': (-1.0, 'Gol sofrido'),
        'scout.FC': (-0.3, 'Falta cometida'),
        'scout.PC': (-1.0, 'Pênalti cometido'),
        'scout.GC': (-3.0, 'Gol contra'),
        'scout.CV': (-3.0, 'Cartão vermelho')
    }
    
    return pontuacao[scout][0]

In [3]:
# Verificacoes

def verifica_dados(df_atletas_flat, numero_rodada, total_atletas_rodada):
    ano = datetime.date.today().year
    
    #verifica se o numero de atletas esta ok
    if df_atletas_flat.shape[0] != total_atletas_rodada:
        print('Erro no numero de atletas da rodada! apurado:{}, indicado:{}'.format(df_atletas_flat.shape[0], total_atletas_rodada))
        return False
    #else:
    #    print('Verificacao numero de atletas da rodada ok! apurado:{}, indicado:{}'.format(df_atletas_flat.shape[0], total_atletas_rodada))

    #verifica se possui todos os scouts
    for i in scouts_possiveis():
        if i not in df_atletas_flat.columns:
            #print('Coluna {} nao encontrada. Adicionando'.format(i))
            df_atletas_flat[i] = 0

    #verifica se a pontuacao dos atletas esta ok
    df_atletas_flat['pontuacao_calculada']=\
        df_atletas_flat['scout.G']  * get_pontuacao_scouts('scout.G')  +\
        df_atletas_flat['scout.A']  * get_pontuacao_scouts('scout.A')  +\
        df_atletas_flat['scout.FT'] * get_pontuacao_scouts('scout.FT') +\
        df_atletas_flat['scout.FD'] * get_pontuacao_scouts('scout.FD') +\
        df_atletas_flat['scout.FF'] * get_pontuacao_scouts('scout.FF') +\
        df_atletas_flat['scout.FS'] * get_pontuacao_scouts('scout.FS') +\
        df_atletas_flat['scout.PS'] * get_pontuacao_scouts('scout.PS') +\
        df_atletas_flat['scout.PP'] * get_pontuacao_scouts('scout.PP') +\
        df_atletas_flat['scout.I']  * get_pontuacao_scouts('scout.I')  +\
        df_atletas_flat['scout.PI'] * get_pontuacao_scouts('scout.PI') +\
        df_atletas_flat['scout.DP'] * get_pontuacao_scouts('scout.DP') +\
        df_atletas_flat['scout.SG'] * get_pontuacao_scouts('scout.SG') +\
        df_atletas_flat['scout.DE'] * get_pontuacao_scouts('scout.DE') +\
        df_atletas_flat['scout.DS'] * get_pontuacao_scouts('scout.DS') +\
        df_atletas_flat['scout.CA'] * get_pontuacao_scouts('scout.CA') +\
        df_atletas_flat['scout.GS'] * get_pontuacao_scouts('scout.GS') +\
        df_atletas_flat['scout.FC'] * get_pontuacao_scouts('scout.FC') +\
        df_atletas_flat['scout.PC'] * get_pontuacao_scouts('scout.PC') +\
        df_atletas_flat['scout.GC'] * get_pontuacao_scouts('scout.GC') +\
        df_atletas_flat['scout.CV'] * get_pontuacao_scouts('scout.CV')

    df_atletas_flat['diff_pontuacao'] = abs(df_atletas_flat['pontuacao_calculada']-df_atletas_flat['pontuacao'])

    numero_atletas_errado = df_atletas_flat[(df_atletas_flat['posicao_id']!=6) & (df_atletas_flat['diff_pontuacao'] > 0.0001)].shape[0]

    if numero_atletas_errado > 0:
        print(df_atletas_flat[(df_atletas_flat['posicao_id']!=6) & (df_atletas_flat['diff_pontuacao'] > 0.0001)].head())
        print('Erro na pontuacao de alguns atletas. Numero de atletas com problema:{}'.format(numero_atletas_errado))
        return False
    #else:
    #    print('Pontuacao dos atletas OK')

    df_atletas_flat['rodada'] = numero_rodada
    df_atletas_flat['ano'] = ano

    # exclui colunas scout, pontuacao_calculada e diff_pontuacao
    df_atletas_flat = df_atletas_flat[['apelido', 'foto', 'pontuacao', 'posicao_id', 'clube_id',\
           'entrou_em_campo', 'id_atleta', 'scout.FC', 'scout.FS', 'scout.PI',\
           'scout.FF', 'scout.A', 'scout.DS', 'scout.DE', 'scout.GS', 'scout.FD',\
           'scout.SG', 'scout.CA', 'scout.FT', 'scout.G', 'scout.PC',\
           'scout.PS', 'scout.PP', 'scout.I', 'scout.DP', 'scout.GC', 'scout.CV',\
           'rodada', 'ano']]
    return True, df_atletas_flat

In [5]:
def create_scouts_dataframe(last_turn):
    df_all = pd.DataFrame()

    for num_rodada in range(1, last_turn):
        #print('Processando rodada={}'.format(num_rodada))
        df_atletas_flat, numero_rodada, total_atletas_rodada = get_dados_rodada(num_rodada)
        flagVerificao, df_atletas_flat = verifica_dados(df_atletas_flat, numero_rodada, total_atletas_rodada)
        if not flagVerificao:
            print('================= Dados nao confiaveis ===================')
        else:
            df_all = df_all.append(df_atletas_flat, ignore_index = True)
            
    df_scouts = df_all[['apelido', 'pontuacao', 'posicao_id', 'clube_id',\
       'entrou_em_campo', 'id_atleta', 'scout.FC', 'scout.FS', 'scout.PI',\
       'scout.FF', 'scout.A', 'scout.DS', 'scout.DE', 'scout.GS', 'scout.FD',\
       'scout.SG', 'scout.CA', 'scout.FT', 'scout.G', 'scout.PC', 'scout.PS',\
       'scout.PP', 'scout.I', 'scout.DP', 'scout.GC', 'scout.CV', 'rodada', 'ano']]
    
    df_scouts.columns = ['player_name', 'points', 'id_position', 'id_club',\
                    'has_played', 'id_player', 'scout_fc', 'scout_fs', 'scout_pi',\
       'scout_ff', 'scout_a', 'scout_ds', 'scout_de', 'scout_gs', 'scout_fd',\
       'scout_sg', 'scout_ca', 'scout_ft', 'scout_g', 'scout_pc', 'scout_ps',\
       'scout_pp', 'scout_i', 'scout_dp', 'scout_gc', 'scout_cv', 'round', 'year']
    
    return df_scouts

In [7]:
def collect_scouts_and_save_to_database(engine):
    last_turn = request_last_turn_from_cartola_api()
    df_scouts = create_scouts_dataframe(last_turn)
    save_data(engine, 'scouts', df_scouts)
    return count_data(engine, 'scouts').fetchall()